In [3]:
import pandas as pd
import gc

prev_app = pd.read_csv("input/previous_application.csv")

prev_app['DAYS_FIRST_DRAWING']        = prev_app['DAYS_FIRST_DRAWING'].fillna(365.243)
prev_app['DAYS_LAST_DUE_1ST_VERSION'] = prev_app['DAYS_FIRST_DRAWING'].fillna(365.243)
prev_app['DAYS_TERMINATION']          = prev_app['DAYS_FIRST_DRAWING'].fillna(365.243)
prev_app['DAYS_LAST_DUE']             = prev_app['DAYS_FIRST_DRAWING'].fillna(365.243)
# Add feature: value ask / value received percentage
prev_app['APP_CREDIT_PERC']           = prev_app['AMT_APPLICATION'] / prev_app['AMT_CREDIT']

In [4]:
old_columns         = prev_app.columns
numerical_columns   = [col for col in prev_app.columns if prev_app[col].dtype != 'object']
categorical_columns = [col for col in prev_app.columns if prev_app[col].dtype == 'object']
prev_app            = pd.get_dummies(prev_app,columns=categorical_columns)
new_columns         = prev_app.columns
incremental         = set(new_columns) - set(old_columns)
incremental         = list(incremental)
cat_aggregations = {}
for cat in incremental:
    cat_aggregations[cat] = ['mean']

num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum']
        }

prev_agg            = prev_app.groupby('SK_ID_CURR').agg({**num_aggregations,**cat_aggregations})
prev_agg.columns    = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
prev_agg.reset_index(inplace=True)

approved             = prev_app[prev_app['NAME_CONTRACT_STATUS_Approved'] == 1]
approved_agg         = approved.groupby('SK_ID_CURR').agg(num_aggregations)
approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
approved_agg.reset_index(inplace=True)
prev_agg             = pd.merge(prev_agg,approved_agg, how='left', on='SK_ID_CURR')


# Previous Applications: Refused Applications - only numerical features
refused = prev_app[prev_app['NAME_CONTRACT_STATUS_Refused'] == 1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
refused_agg.reset_index(inplace=True)
prev_agg = pd.merge(prev_agg,refused_agg, how='left', on='SK_ID_CURR')

In [5]:
prev_agg.to_pickle("prev_data.p")
del refused, refused_agg, approved, approved_agg, prev_app
gc.collect()

1917